Архитектура модели анализа кода

В данном файле проводится анализ архитектуры модели, токенизатора и подготовка к обучению модели

Импортируем необходимые модули

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import re

import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

from torch.utils.tensorboard import summary, writer, SummaryWriter
from tqdm import tqdm
import time
import datetime

Устанавливаем SEED

In [2]:
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

import warnings
warnings.filterwarnings("ignore")

Далее считываем исходный датасет и немного дорабатываем его

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
dataset_path = '/Users/chervonikov_alexey/Desktop/projects/Technopark_Autumn_2024/NN_course_project/model/upd_code_dataset.parquet'

In [5]:
code_dataset = pd.read_parquet(dataset_path)

In [6]:
code_dataset.head()

,response,focal_method,focal_cls,focal_method_ast,focal_cls_ast,focal_method_info,focal_cls_info,input_string_focal_method,input_string_focal_cls
0,"from microdot import Microdot, Response, abort...","<FUNC_TOKEN> def get(self, key, default=None):...",<CLS_TOKEN> <FUNC_TOKEN>,<AST_TOKEN> Module( body=[ FunctionDef( name='...,<AST_TOKEN>,<INFO_TOKEN>,<INFO_TOKEN>,"<FUNC_TOKEN> def get(self, key, default=None):...",<CLS_TOKEN> <FUNC_TOKEN> <INFO_TOKEN> <AST_TOKEN>
1,"from microdot import Microdot, Response, abort...","<FUNC_TOKEN> def get(self, url_pattern): retur...","<CLS_TOKEN> class Microdot: def route(self, ur...",<AST_TOKEN> Module( body=[ FunctionDef( name='...,<AST_TOKEN> Module( body=[ ClassDef( name='Mic...,<INFO_TOKEN> <DESCRIPTION_TOKEN> Decorator tha...,<INFO_TOKEN> Module( body=[ ClassDef( name='Mi...,"<FUNC_TOKEN> def get(self, url_pattern): retur...","<CLS_TOKEN> class Microdot: def route(self, ur..."
2,"from microdot import Microdot, Response, abort...","<FUNC_TOKEN> def post(self, url_pattern): retu...","<CLS_TOKEN> class Microdot: def route(self, ur...",<AST_TOKEN> Module( body=[ FunctionDef( name='...,<AST_TOKEN> Module( body=[ ClassDef( name='Mic...,<INFO_TOKEN> <DESCRIPTION_TOKEN> Decorator tha...,<INFO_TOKEN> Module( body=[ ClassDef( name='Mi...,"<FUNC_TOKEN> def post(self, url_pattern): retu...","<CLS_TOKEN> class Microdot: def route(self, ur..."
3,"from microdot import Microdot, Response, abort...","<FUNC_TOKEN> def mount(self, subapp, url_prefi...",<CLS_TOKEN> <FUNC_TOKEN>,<AST_TOKEN> Module( body=[ FunctionDef( name='...,<AST_TOKEN>,<INFO_TOKEN> <DESCRIPTION_TOKEN> Mount a sub-a...,<INFO_TOKEN>,"<FUNC_TOKEN> def mount(self, subapp, url_prefi...",<CLS_TOKEN> <FUNC_TOKEN> <INFO_TOKEN> <AST_TOKEN>
4,from pyner.named_entity.corpus import bio2bioe...,<FUNC_TOKEN> def iob2bio(tags): processed_tags...,<CLS_TOKEN> def split_tag(tag: str): if tag in...,<AST_TOKEN> Module( body=[ FunctionDef( name='...,<AST_TOKEN> Module( body=[ FunctionDef( name='...,<INFO_TOKEN> <DESCRIPTION_TOKEN> should be bio...,<INFO_TOKEN> Module( body=[ FunctionDef( name=...,<FUNC_TOKEN> def iob2bio(tags): processed_tags...,<CLS_TOKEN> def split_tag(tag: str): if tag in...


In [7]:
code_dataset = code_dataset.reset_index(drop=True)

Наконец, переходим к анализу архитектур нейросетей

Решено использовать подход, основанный на обучении (fine-tuning) нейросети CodeBERT, в основе которой лежит модель RoBERTa. Далее будем использовать метамодель в виде декодера (CodeGen или GPTBigCode)

In [8]:
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM

Device:

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [10]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

zsh:1: command not found: nvidia-smi


In [11]:
from transformers import GPT2LMHeadModel, AutoConfig

class LargeCodeModelGPTBigCode(nn.Module):
	'''Класс для сложной языковой модели, которая обрабатывает входной код'''
	def __init__(self, gpt2_name):
		super(LargeCodeModelGPTBigCode, self).__init__()

		self.new_special_tokens = ['<FUNC_TOKEN>',
            '<INFO_TOKEN>',
            '<CLS_TOKEN>',
            '<AST_TOKEN>',
            '<DESCRIPTION_TOKEN>',
            '<COMMENTS_TOKEN>']

		self.special_tokens_dict = {
			'additional_special_tokens': self.new_special_tokens
		}

		self.tokenizerGPT = AutoTokenizer.from_pretrained(gpt2_name)
		self.tokenizerGPT.add_special_tokens({'pad_token': '<PAD>'})
		self.tokenizerGPT.add_special_tokens(self.special_tokens_dict)
		self.gpt2 = AutoModelForCausalLM.from_pretrained(gpt2_name)
		self.gpt2.resize_token_embeddings(len(self.tokenizerGPT))

	# forward call
	def forward(self, input_ids, attention_mask,
			 	response_ids, response_attention_mask):
		

		gpt2_outputs = self.gpt2(input_ids = input_ids,
						   attention_mask = attention_mask,
						   labels = response_ids)

		return gpt2_outputs


In [12]:
CodeModel = LargeCodeModelGPTBigCode(gpt2_name="bigcode/gpt_bigcode-santacoder").to(device)

In [13]:
code_dataset.head()

,response,focal_method,focal_cls,focal_method_ast,focal_cls_ast,focal_method_info,focal_cls_info,input_string_focal_method,input_string_focal_cls
0,"from microdot import Microdot, Response, abort...","<FUNC_TOKEN> def get(self, key, default=None):...",<CLS_TOKEN> <FUNC_TOKEN>,<AST_TOKEN> Module( body=[ FunctionDef( name='...,<AST_TOKEN>,<INFO_TOKEN>,<INFO_TOKEN>,"<FUNC_TOKEN> def get(self, key, default=None):...",<CLS_TOKEN> <FUNC_TOKEN> <INFO_TOKEN> <AST_TOKEN>
1,"from microdot import Microdot, Response, abort...","<FUNC_TOKEN> def get(self, url_pattern): retur...","<CLS_TOKEN> class Microdot: def route(self, ur...",<AST_TOKEN> Module( body=[ FunctionDef( name='...,<AST_TOKEN> Module( body=[ ClassDef( name='Mic...,<INFO_TOKEN> <DESCRIPTION_TOKEN> Decorator tha...,<INFO_TOKEN> Module( body=[ ClassDef( name='Mi...,"<FUNC_TOKEN> def get(self, url_pattern): retur...","<CLS_TOKEN> class Microdot: def route(self, ur..."
2,"from microdot import Microdot, Response, abort...","<FUNC_TOKEN> def post(self, url_pattern): retu...","<CLS_TOKEN> class Microdot: def route(self, ur...",<AST_TOKEN> Module( body=[ FunctionDef( name='...,<AST_TOKEN> Module( body=[ ClassDef( name='Mic...,<INFO_TOKEN> <DESCRIPTION_TOKEN> Decorator tha...,<INFO_TOKEN> Module( body=[ ClassDef( name='Mi...,"<FUNC_TOKEN> def post(self, url_pattern): retu...","<CLS_TOKEN> class Microdot: def route(self, ur..."
3,"from microdot import Microdot, Response, abort...","<FUNC_TOKEN> def mount(self, subapp, url_prefi...",<CLS_TOKEN> <FUNC_TOKEN>,<AST_TOKEN> Module( body=[ FunctionDef( name='...,<AST_TOKEN>,<INFO_TOKEN> <DESCRIPTION_TOKEN> Mount a sub-a...,<INFO_TOKEN>,"<FUNC_TOKEN> def mount(self, subapp, url_prefi...",<CLS_TOKEN> <FUNC_TOKEN> <INFO_TOKEN> <AST_TOKEN>
4,from pyner.named_entity.corpus import bio2bioe...,<FUNC_TOKEN> def iob2bio(tags): processed_tags...,<CLS_TOKEN> def split_tag(tag: str): if tag in...,<AST_TOKEN> Module( body=[ FunctionDef( name='...,<AST_TOKEN> Module( body=[ FunctionDef( name='...,<INFO_TOKEN> <DESCRIPTION_TOKEN> should be bio...,<INFO_TOKEN> Module( body=[ FunctionDef( name=...,<FUNC_TOKEN> def iob2bio(tags): processed_tags...,<CLS_TOKEN> def split_tag(tag: str): if tag in...


In [14]:
code_dataset['input_string'] = code_dataset['input_string_focal_method'] + ' ' + code_dataset['input_string_focal_cls']

In [15]:
print(code_dataset['input_string'].values[0])

<FUNC_TOKEN> def get(self, key, default=None): kl = key.lower() return super().get(self.keymap.get(kl, kl), default) <INFO_TOKEN> <AST_TOKEN> Module( body=[ FunctionDef( name='get', args=arguments( posonlyargs=[], args=[ arg(arg='self'), arg(arg='key'), arg(arg='default')], kwonlyargs=[], kw_defaults=[], defaults=[ Constant(value=None)]), body=[ Assign( targets=[ Name(id='kl', ctx=Store())], value=Call( func=Attribute( value=Name(id='key', ctx=Load()), attr='lower', ctx=Load()), args=[], keywords=[])), Return( value=Call( func=Attribute( value=Call( func=Name(id='super', ctx=Load()), args=[], keywords=[]), attr='get', ctx=Load()), args=[ Call( func=Attribute( value=Attribute( value=Name(id='self', ctx=Load()), attr='keymap', ctx=Load()), attr='get', ctx=Load()), args=[ Name(id='kl', ctx=Load()), Name(id='kl', ctx=Load())], keywords=[]), Name(id='default', ctx=Load())], keywords=[]))], decorator_list=[], type_params=[])], type_ignores=[]) <CLS_TOKEN> <FUNC_TOKEN> <INFO_TOKEN> <AST_TOKEN

Далее необхоимо описать класс Dataset для нашей модели

In [16]:
class Code2TestDataset(Dataset):
	'''Класс датасет для задачи генерации тестов'''

	def __init__(self, code_dataset, tokenizer_gpt, max_length=1024):
		'''
		Конструктор датасета

		Параметры:
		- code_dataset: датасет pd.DataFrame
		- tokenizer_gpt: токенизатор gpt
		- max_length: максимальная длина последовательности (default: 1024)
		'''
		self.code_dataset = code_dataset
		self.tokenizer_gpt = tokenizer_gpt
		self.max_length = max_length

	def __getitem__(self, idx, idx_to_token=False):
		'''
		Get-метод - возвращает сэмпл по индексу

		Параметры:
		- idx: индекс
		- idx_to_token: флаг для отображения токенов из индексов (default: False)
		'''
		input = self.code_dataset.at[idx, 'input_string']
		response = self.code_dataset.at[idx, 'response']

		def encode_text(text, tokenizer):
			encoding = tokenizer.encode_plus(
				text,
				add_special_tokens=True,
				max_length=self.max_length,
				padding='max_length',
				truncation=True,
				return_attention_mask=True,
				return_tensors='pt',
			)
			input_ids = encoding['input_ids'].flatten()
			attention_mask = encoding['attention_mask'].flatten()
			return input_ids, attention_mask

		input_ids, attention_mask = encode_text(input, self.tokenizer_gpt)
		input_ids_response, attention_mask_response = encode_text(response, self.tokenizer_gpt)

		if idx_to_token:
			return {
					 'input_ids': self.tokenizer_gpt.convert_ids_to_tokens(input_ids),
					 'attention_mask': attention_mask,
					 'input_ids_response': self.tokenizer_gpt.convert_ids_to_tokens(input_ids_response),
					 'attention_mask_response': attention_mask_response
				# 'input_ids_focal_method': self.tokenizer_code_bert.convert_ids_to_tokens(input_ids_focal_method),
				# 'attention_mask_focal_method': attention_mask_focal_method,
				# 'input_ids_focal_cls': self.tokenizer_code_bert.convert_ids_to_tokens(input_ids_focal_cls),
				# 'attention_mask_focal_cls': attention_mask_focal_cls,
				# 'ids_response': self.tokenizer_gpt.convert_ids_to_tokens(input_ids_response),
				# 'attention_mask_response': attention_mask_response,
				# 'input_ids_focal_method_decoder': self.tokenizer_gpt.convert_ids_to_tokens(input_ids_focal_method_decoder),
				# 'attention_mask_focal_method_decoder': attention_mask_focal_method_decoder,
				# 'input_ids_focal_cls_decoder': self.tokenizer_gpt.convert_ids_to_tokens(input_ids_focal_cls_decoder),
				# 'attention_mask_focal_cls_decoder': attention_mask_focal_cls_decoder
			}
		return {
				 'input_ids': input_ids,
				'attention_mask': attention_mask,
				'input_ids_response': input_ids_response,
				'attention_mask_response': attention_mask_response
}

	def __len__(self):
		'''Функция возвращает длину датасета. В качестве длины берется размер датасета по axis = 0'''
		return self.code_dataset.shape[0]


Тестируем написанный класс

In [17]:
code2test_dataset = Code2TestDataset(code_dataset=code_dataset,
                                     tokenizer_gpt=CodeModel.tokenizerGPT)

In [18]:
print(code2test_dataset.__getitem__(0, True)['input_ids'])

['<FUNC_TOKEN>', 'Ġdef', 'Ġget', '(', 'self', ',', 'Ġkey', ',', 'Ġdefault', '=', 'None', '):', 'Ġkl', 'Ġ=', 'Ġkey', '.', 'lower', '()', 'Ġreturn', 'Ġsuper', '().', 'get', '(', 'self', '.', 'keymap', '.', 'get', '(', 'kl', ',', 'Ġkl', '),', 'Ġdefault', ')', 'Ġ', '<INFO_TOKEN>', 'Ġ', '<AST_TOKEN>', 'ĠModule', '(', 'Ġbody', '=[', 'ĠFunction', 'Def', '(', 'Ġname', "='", 'get', "',", 'Ġargs', '=', 'arguments', '(', 'Ġpos', 'only', 'args', '=[],', 'Ġargs', '=[', 'Ġarg', '(', 'arg', "='", 'self', "'),", 'Ġarg', '(', 'arg', "='", 'key', "'),", 'Ġarg', '(', 'arg', "='", 'default', "')],", 'Ġkw', 'only', 'args', '=[],', 'Ġkw', '_', 'defaults', '=[],', 'Ġdefaults', '=[', 'ĠConstant', '(', 'value', '=', 'None', ')]),', 'Ġbody', '=[', 'ĠAssign', '(', 'Ġtargets', '=[', 'ĠName', '(', 'id', "='", 'kl', "',", 'Ġctx', '=', 'Store', '())', '],', 'Ġvalue', '=', 'Call', '(', 'Ġfunc', '=', 'Attribute', '(', 'Ġvalue', '=', 'Name', '(', 'id', "='", 'key', "',", 'Ġctx', '=', 'Load', '()),', 'Ġattr', "='", 'low

In [19]:
print(code2test_dataset.__getitem__(0, True)['input_ids_response'])

['from', 'Ġmicro', 'dot', 'Ġimport', 'ĠMicro', 'dot', ',', 'ĠResponse', ',', 'Ġabort', 'Ċ', 'from', 'Ġmicro', 'dot', '.', 'test', '_', 'client', 'Ġimport', 'ĠTest', 'Client', 'Ċ', 'Ċ', 'class', 'ĠTest', 'Micro', 'dot', '(', 'unittest', '.', 'TestCase', '):', 'ĊĠĠĠ', 'Ġdef', 'Ġ_', 'run', '(', 'self', ',', 'Ġcor', 'o', '):', 'ĊĠĠĠĠĠĠĠ', 'Ġreturn', 'Ġself', '.', 'loop', '.', 'run', '_', 'until', '_', 'complete', '(', 'coro', ')', 'ĊĠĠĠ', 'Ġdef', 'Ġtest', '_', 'get', '_', 'request', '(', 'self', '):', 'ĊĠĠĠĠĠĠĠ', 'Ġapp', 'Ġ=', 'ĠMicro', 'dot', '()', 'ĊĠĠĠĠĠĠĠ', 'Ġ@', 'app', '.', 'route', "('/')", 'ĊĠĠĠĠĠĠĠ', 'Ġdef', 'Ġindex', '(', 'req', '):', 'ĊĠĠĠĠĠĠĠĠĠĠĠ', 'Ġreturn', "Ġ'", 'foo', "'", 'ĊĠĠĠĠĠĠĠ', 'Ġ@', 'app', '.', 'route', "('/", 'async', "')", 'ĊĠĠĠĠĠĠĠ', 'Ġasync', 'Ġdef', 'Ġindex', '2', '(', 'req', '):', 'ĊĠĠĠĠĠĠĠĠĠĠĠ', 'Ġreturn', "Ġ'", 'foo', '-', 'async', "'", 'ĊĠĠĠĠĠĠĠ', 'Ġ@', 'app', '.', 'route', "('/", 'arg', '/<', 'id', ">')", 'ĊĠĠĠĠĠĠĠ', 'Ġdef', 'Ġindex', '3', '(', 'req', ',', 

In [20]:
code2test_dataset[0]['attention_mask']

tensor([1, 1, 1,  ..., 0, 0, 0])

In [21]:
code2test_dataset[0]['attention_mask_response']

tensor([1, 1, 1,  ..., 0, 0, 0])

In [22]:
print(f"Длина датасета составляет: {len(code2test_dataset)}")

Длина датасета составляет: 280458


Всё работает корректно! Следующим шагом необходимо разбить датасет на train и val

In [23]:
def get_datasets(dataset_cls = Code2TestDataset,
				max_length = 1024,
				data = code_dataset,
				tokenizer_gpt = CodeModel.tokenizerGPT,
				train_size = 0.7):
	'''
	Функция get_datasets() возвращает train и val датасеты на основе конструктора AccentDataset, делая train_val_spilt

	Параметры:
	-dataset_cls: класс датасета, конструктор которого будет вызываться (default: Code2TestDataset)
	-max_length: максимальная статья последовательности токенов
	-data: датасает pd.DataFrame (default: code_dataset)
	-tokenizer_code_bert: токенизатор codeBERT (default: tokenizer_code_bert)
	-tokenizer_gpt: токенизатор GPT2 (default: tokenizer_gpt)
	-train_size: размер тренировочной выборки (default: 0.7)

	'''

	dataset = dataset_cls(code_dataset = data,
						tokenizer_gpt=tokenizer_gpt,
						max_length=max_length)

	train_size = int(train_size * len(dataset))
	val_size = len(dataset) - train_size
	train_dataset, test_dataset = random_split(dataset, [train_size, val_size])

	return train_dataset, test_dataset

train_dataset, val_dataset = get_datasets(train_size=0.9)

Проверяем полученные датасеты

In [24]:
print(f"Количество данных в train и val выборках соответственно: {len(train_dataset), len(val_dataset)}")

Количество данных в train и val выборках соответственно: (252412, 28046)


In [25]:
def decode_sequence(tokens_ids, tokenizer):
	'''Декодирование последовательности токенов'''
	code_bert_decoded = tokenizer.decode(tokens_ids)
	print(f"Декодированная строка: {code_bert_decoded}")

Далее получим DataLoader, по которому будем итерироваться

In [26]:
def get_loaders(train_dataset = train_dataset,
			val_dataset = val_dataset,
			shuffle_train = True,
			shuffle_val = False,
			batch_size = 32):

	'''
	Функция get_loaders() для получения train, val даталоадеров

	Параметры:
	-train_dataset: тренировочный датасет (default: train_dataset)
	-val_dataset: валидационный датасет (default: val_dataset)
	-shuffle_train: флаг перемешивания для train (default: True)
	-shuffle_val: флаг перемешивания для val (default: False)
	-batch_size: размер батча данных (default: 32)
	'''

	# train_dataloader
	train_dataloader = DataLoader(
			train_dataset,
			batch_size = batch_size,
			shuffle = shuffle_train,
		)

	# validation_dataloader
	validation_dataloader = DataLoader(
			val_dataset,
			batch_size = batch_size,
			shuffle = shuffle_val,
		)

	# Возвращаем даталоадеры
	return train_dataloader, validation_dataloader

train_dataloader, validation_dataloader = get_loaders(batch_size=4)

Проверка итерирования

In [28]:
for i, batch in enumerate(tqdm(train_dataloader)):
    if i == 0:
        break
    pass

  0%|          | 0/63103 [00:00<?, ?it/s]


Корректно отрабатывает!

Далее, собираем архитектуру и готовимся обучать

In [29]:
# model_code_bert = AutoModel.from_pretrained("microsoft/codebert-base", output_hidden_states= True).to(device)
# model_code_bert.resize_token_embeddings(len(tokenizer_code_bert))

Как работает модель codeBERT:

In [30]:
# for i, batch in enumerate(train_dataloader):

# 	# Проверка корректности работы
# 	b_input_ids = batch['input_ids_focal_method'].to(device)
# 	b_input_mask = batch['attention_mask_focal_method'].to(device)

# 	outputs_code_bert = model_code_bert(b_input_ids, attention_mask=b_input_mask)
# 	last_hidden_state_code_bert = outputs_code_bert['last_hidden_state']
# 	print(last_hidden_state_code_bert.size())
# 	break

Таким образом, для каждого токена мы получим свое закодированное значение размерности 768

Модель GPT2:

In [31]:
# from transformers import AutoConfig

# modelGPT2Path = "gpt2"
# # config = AutoConfig.from_pretrained(modelGPT2Path, is_decoder=True, add_cross_attention= True)
# # config.add_cross_attention = True  # Включение cross-attention

# # modelGPT2 = AutoModel.from_pretrained(modelGPT2Path, config=config).to(device)
# # modelGPT2.resize_token_embeddings(len(tokenizerGPT))

Как работает модель GPTBigCode

In [32]:
# for i, batch in enumerate(train_dataloader):

# 	b_input_ids = batch['input_ids_focal_method'].to(device)
# 	b_input_mask = batch['attention_mask_focal_method'].to(device)

# 	outputs_code_bert = model_code_bert(b_input_ids, attention_mask=b_input_mask)
# 	last_hidden_state_code_bert = outputs_code_bert['last_hidden_state']

# 	print(last_hidden_state_code_bert.size())

# 	# Проверка корректности работы
# 	response_input_ids = batch['ids_response'].to(device)
# 	response_input_mask = batch['attention_mask_response'].to(device)
# 	gpt_output = modelGPT2(input_ids=response_input_ids,
# 							  attention_mask=response_input_mask,
# 							  encoder_hidden_states = last_hidden_state_code_bert)
# 	print(gpt_output['last_hidden_state'].size())


# 	# outputs_code_bert = model_code_bert(b_input_ids, attention_mask=b_input_mask)
# 	# last_hidden_state_code_bert = outputs_code_bert['last_hidden_state']
# 	# print(last_hidden_state_code_bert.size())
# 	break

Ну, как-то худо-бедно всё это дело запускается. Пробуем строить модель

Отлаживаем модель

Далее необходимо объявить функцию train-val loop

Для начала необходимо объявить дополнительные функции для отображения времени и подсчёта метрик качества

In [102]:
def format_time(elapsed):
	'''Функция форматирования времени'''
	return str(datetime.timedelta(seconds=int(round((elapsed)))))

# def token_accuracy_calc(logits, labels, attention_mask):
# 	'''Функция подсчета accuracy для данных'''
# 	pred_flat = np.argmax(logits, axis=-1).flatten()
# 	# print(pred_flat)
# 	labels_flat = labels.flatten()
# 	# print(labels_flat)
# 	mask_flat = attention_mask.flatten()
# 	pred_flat = pred_flat[mask_flat == 1]
# 	labels_flat = labels_flat[mask_flat == 1]

# 	print(decode_sequence(pred_flat, CodeModel.tokenizerGPT))
# 	print(decode_sequence(labels_flat, CodeModel.tokenizerGPT))
# 	accuracy = np.sum(pred_flat == labels_flat) / len(labels_flat)
# 	return accuracy

def token_bleu_calc(logits, labels, attention_mask):
	'''Функция подсчета BLEU для данных'''

	pred_flat = np.argmax(logits, axis=-1)
	labels_flat = labels
	mask_flat = attention_mask

	pred_tokens = []
	true_tokens = []

	for i in range(pred_flat.shape[0]):
		pred_seq = pred_flat[i]
		true_seq = labels_flat[i]
		pred_tokens.append(pred_seq)
		true_tokens.append(true_seq)


	# print(pred_tokens, true_tokens)
	pred_strings = CodeModel.tokenizerGPT.batch_decode(pred_tokens, skip_special_tokens=True)
	true_strings = CodeModel.tokenizerGPT.batch_decode(true_tokens, skip_special_tokens=True)

	# print(pred_strings, true_strings)

	# Вычисление BLEU
	bleu_scores = []
	smoothing_function = SmoothingFunction().method1

	for pred, true in zip(pred_strings, true_strings):
		pred_tokens = pred.split()
		true_tokens = [true.split()]
		bleu_score = sentence_bleu(true_tokens, pred_tokens, smoothing_function=smoothing_function)
		bleu_scores.append(bleu_score)

	average_bleu = np.mean(bleu_scores)
	return average_bleu



Как минимимум оно запускается

Далее реализуем саму функцию train-val-loop

Перед этим объями дополнительные настройки обучения

In [ ]:
from transformers import AdamW, get_scheduler

optimizer = AdamW(CodeModel.parameters(), lr=3e-5)
num_epochs = 1
train_steps = len(train_dataloader) * num_epochs
lr_scheduler = get_scheduler(
    name='linear',
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=train_steps
)
tensorboard_log_dir = 'runs/CodeModelLogs/'
tensorboard_path_loss = 'runs/model_val_loss.pth'
tensorboard_path_bleu = 'runs/model_bleu_score.pth'

И, наконец, функция:

In [ ]:
def train_val_loop_codeLM(model = CodeModel,
						train_loader = train_dataloader,
						val_loader = validation_dataloader,
						optimizer = optimizer,
						scheduler = lr_scheduler,
						num_epochs = num_epochs,
						device = 'cuda',
						model_save_path_loss = tensorboard_path_loss,
						model_save_path_bleu = tensorboard_path_bleu,
						tensorboard_log_dir = tensorboard_log_dir,
						gradient_accumulation_steps = 2,
						eval_every = 1,
						test_step_only = False):
	'''
	Функция для реализации train-val loop обучения нашей модели

	Параметры:
	-model: модель нейронной сети
	-train_loader: тренировочный датасет
	-val_loader: валидационный датасет
	-optimizer: оптимизатор
	-scheduler: изменение для learning_rate (расписание)
	-num_epochs: число эпох для обучения
	-device: устройство
	-model_save_path_loss: путь для сохранения весов модели (с лучим val_loss)
	-model_save_path_bleu: путь для сохранения весов модели (с лучим val_bleu_score)
	-tensorboard_log_dir: путь для записи логов в TensorBoard,
	-gradient_accumulation_steps: число шагов для накопления градиентов
	-eval_every: число шагов, через которые делаем валидацию
	-test_step_only: вспомогательная логика для тестирования обучения небольшого числа шагов (default: False)
	'''

	writer = SummaryWriter(log_dir=tensorboard_log_dir)
	history = {
		'train_loss': [],
		'train_bleu': [],
		'val_loss': [],
		'val_bleu': []
	}
	best_val_loss = float('inf')
	best_val_bleu = 0.0
	best_val_accuracy = 0.0
	model.to(device)

	for epoch in range(num_epochs):
		print("")
		print('======== Epoch {:} / {:} ========'.format(epoch + 1, num_epochs))
		print('Training...')

		t0 = time.time()
		model.train()
		total_train_loss = 0
		total_train_bleu = 0
		num_train_steps = 0

		for step, batch in enumerate(tqdm(train_loader)):

			if test_step_only and step >= 800:  # Прерываем после первого батча
				break


			if step % 1500 == 0 and not step == 0:
				# Calculate elapsed time in minutes.
				elapsed = format_time(time.time() - t0)
				# Report progress.
				print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

			optimizer.zero_grad()

			input_ids = batch['input_ids'].to(device)
			attention_masks = batch['attention_mask'].to(device)

			response_ids = batch['input_ids_response']
			response_attention_masks = batch['attention_mask_response']
			
			output_codeLM = model(input_ids, attention_masks, 
						 			response_ids, response_attention_masks)

			loss = output_codeLM.loss
			logits = output_codeLM.logits

			logits = logits.detach().cpu().numpy()
			response_ids = response_ids.cpu().numpy()
			response_attention_masks = response_attention_masks.cpu().numpy()

			bleu_train = token_bleu_calc(logits, response_ids, response_attention_masks)

			total_train_bleu += bleu_train

			total_train_loss += loss.item()
			num_train_steps += 1

			loss.backward()

			if (step + 1) % gradient_accumulation_steps == 0 or (step + 1) == len(train_dataloader):
				torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # Клипаем накопленные градиенты
				optimizer.step()
				scheduler.step()

		avg_train_loss = total_train_loss / num_train_steps
		avg_train_bleu_score = total_train_bleu / num_train_steps

		training_time = format_time(time.time() - t0)

		print("")
		print("  Average training loss: {0:.2f}".format(avg_train_loss))
		# print("  Average training accuracy: {0:.2f}".format(avg_train_accuracy))
		# print("  Average training BLEU score: {0:.2f}".format(avg_train_bleu_score))
		print("  Training epoch took: {:}".format(training_time))

		history['train_loss'].append(avg_train_loss)
		history['train_bleu'].append(avg_train_bleu_score)

		# Логирование в TensorBoard для обучения
		writer.add_scalar("Train/Loss", avg_train_loss, epoch + 1)
		writer.add_scalar("Train/BLEU_score", avg_train_bleu_score, epoch + 1)

		print(f"Train Loss: {avg_train_loss:.4f}, Train BLEU score: {avg_train_bleu_score:.4f}")

		print("")
		print("Running Validation...")


		t0 = time.time()

		# Put the model in evaluation mode--the dropout layers behave differently
		# during evaluation.
		model.eval()

		if (epoch + 1) % eval_every == 0:
			model.eval()
			total_eval_loss = 0
			total_eval_bleu = 0
			num_eval_steps = 0

		with torch.no_grad():
			for batch in tqdm(val_loader):
				if test_step_only and num_eval_steps >= 200:  # Прерываем если хотим проконтроллировать
					break

				input_ids = batch['input_ids'].to(device)
				attention_masks = batch['attention_mask'].to(device)

				response_ids = batch['input_ids_response']
				response_attention_masks = batch['attention_mask_response']

				output_codeLM = model(input_ids, attention_masks,
						  response_ids, response_attention_masks)

				loss = output_codeLM.loss
				logits = output_codeLM.logits

				logits = logits.detach().cpu().numpy()
				response_ids = response_ids.cpu().numpy()
				response_attention_masks = response_attention_masks.cpu().numpy()

				bleu_val = token_bleu_calc(logits, response_ids, response_attention_masks)

				total_eval_bleu += bleu_val

				total_eval_loss += loss.item()
				num_eval_steps += 1

		avg_val_loss = total_eval_loss / num_eval_steps
		avg_val_bleu_score = total_eval_bleu / num_eval_steps

		history['val_loss'].append(avg_val_loss)
		history['val_bleu'].append(avg_val_bleu_score)

		# Логирование в TensorBoard для валидации
		writer.add_scalar("Validation/Loss", avg_val_loss, epoch + 1)

		print(f"Validation Loss: {avg_val_loss:.4f},  Validation BLEU score: {avg_val_bleu_score:.4f}")

		# Ну вот тут надо настроить, чтобы
		# Если лосс ниже, то сохраняем веса
		if avg_val_loss < best_val_loss:
			best_val_loss = avg_val_loss
			torch.save(model.state_dict(), model_save_path_loss)
			print(f"Model saved to {model_save_path_loss}")

		# Если BLEU выше, то сохраняем веса
		if avg_val_bleu_score > best_val_bleu:
			best_val_bleu = avg_val_bleu_score
			torch.save(model.state_dict(), model_save_path_bleu)
			print(f"Model saved to {model_save_path_bleu}")

	writer.close()
	return history

Наконец, пробуем запустить обучение

In [ ]:
training_results = train_val_loop_codeLM(device='cpu', test_step_only = True, num_epochs = 1)

Подобие инференса

In [99]:
inputs = CodeModel.tokenizerGPT("def is_even(a): if a % 2 == 0: return True return <MASK> ", return_tensors="pt", max_length=30, padding='max_length')
print(CodeModel.tokenizerGPT.convert_ids_to_tokens(inputs['input_ids'][0]))
response = CodeModel.tokenizerGPT(" test_valid(n):  a % 2 == 0", return_tensors='pt', max_length=30, padding='max_length')
with torch.no_grad():

	input_ids, attention_mask = inputs['input_ids'], inputs['attention_mask']
	response_ids, response_attention_mask = response['input_ids'], response['attention_mask']
	outputs = CodeModel(input_ids, attention_mask, 
						response_ids, response_attention_mask)
	loss = outputs.loss
	logits = outputs.logits

	logits = logits.detach().cpu().numpy()
	response_ids = response_ids.cpu().numpy()
	response_attention_masks = response_attention_mask.cpu().numpy()

	pred_flat = np.argmax(logits, axis=-1)

	print("Input:")
	decode_sequence(input_ids[0], CodeModel.tokenizerGPT)

	print("True Response:")
	decode_sequence(response_ids[0], CodeModel.tokenizerGPT)

	# decode_sequence(focal_method_input_ids[0], CodeModel.tokenizer_code_bert)

	print("Prediction:")
	decode_sequence(pred_flat[0], CodeModel.tokenizerGPT)

	print(response_ids[0])
	print(pred_flat[0])


['def', 'Ġis', '_', 'even', '(', 'a', '):', 'Ġif', 'Ġa', 'Ġ%', 'Ġ', '2', 'Ġ==', 'Ġ', '0', ':', 'Ġreturn', 'ĠTrue', 'Ġreturn', 'Ġ<', 'MASK', '>', 'Ġ', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
Input:
Декодированная строка: def is_even(a): if a % 2 == 0: return True return <MASK> <PAD><PAD><PAD><PAD><PAD><PAD><PAD>
True Response:
Декодированная строка:  test_valid(n):  a % 2 == 0<PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>
Prediction:
Декодированная строка:  test_valid(n):
    a % 2 == 0 else return True else Falsea>









[  703    62  1108     7    77   399   207   373   893   207    17   515
   207    15 49157 49157 49157 49157 49157 49157 49157 49157 49157 49157
 49157 49157 49157 49157 49157 49157]
[ 703   62 1108    7   77  399  258  373  893  207   17  515  207   15
  614  363 1201  614 1353   64   29  185  185  185  185  185  185  185
  185  185]


In [100]:
token_bleu_calc(logits, response_ids, response_attention_masks)

0.4617366309441026

In [101]:
token_accuracy_calc(logits, response_ids, attention_mask)

Декодированная строка:  test_valid(n):
    a % 2 == 0 else return True else Falsea>


None
Декодированная строка:  test_valid(n):  a % 2 == 0<PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>
None


1.0